# 第8课：值函数近似（Function Approximation for Value Functions）

> tabular representation to function representation

---

## Part 1：例子 — 曲线拟合（Motivating Example: Function Fitting）

---

### 背景动机

在前面的课程中，我们使用的是表格法（tabular methods）来表示值函数：

* 每个状态 $s$ 对应一个值 $v(s)$；
* 每对状态动作 $(s, a)$ 对应一个值 $q(s, a)$。

**限制：表格法只适用于有限状态空间。**

> 当状态空间很大（甚至是连续空间），表格法就变得不可行 —— 我们需要**函数逼近**方法。

---

### 类比：值函数近似 ≈ 回归问题

我们先看一个更简单的问题：**拟合函数**。

假设我们有若干输入输出对：

$$
(x_1, y_1), (x_2, y_2), \dots, (x_n, y_n)
$$

我们希望学习一个函数 $\hat{f}(x; \theta)$ 来逼近它们，使得：

$$
\hat{f}(x_i; \theta) \approx y_i
$$

这本质上就是一个监督学习（回归）问题。最常见的做法是最小化均方误差（MSE）：

$$
\min_\theta \frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{f}(x_i; \theta) \right)^2
$$

---

### 回到强化学习中的类比

在强化学习中，我们希望：

* 学习一个值函数近似器 $\hat{v}(s; \theta) \approx v^\pi(s)$；
* 或者动作值函数近似器 $\hat{q}(s,a; \theta) \approx q^\pi(s,a)$；

目标是让估计值尽可能接近真实值。

>  **关键点**：我们不能再逐一保存 $v(s)$，而是学习一个通用的函数 $\hat{v}(s; \theta)$ 来对任意 $s$ 预测值。

---

### 举例：用多项式拟合曲线

假设数据点如下：

```text
x:  1   2   3   4   5
y:  2.1 3.9 6.2 8.1 10.0
```

你想找到一个函数 $f(x) = \theta_0 + \theta_1 x$ 来逼近这些点。

我们最小化：

$$
J(\theta) = \sum_{i=1}^n (y_i - (\theta_0 + \theta_1 x_i))^2
$$

最终得到的 $\theta_0, \theta_1$ 可以很好地逼近这些点。

---

### 启示

| 回归问题          | 值函数近似问题                                      |
| ------------- |----------------------------------------------|
| 输入 $x$，输出 $y$ | 输入 $s$（或 $s,a$），输出 $v_\pi(s)$ 或 $q_\pi(s,a)$ |
| 使用损失函数优化参数    | 同样使用损失函数（如 TD 误差）优化参数                        |
| 需要泛化能力        | 值函数也需要在未见过的状态上做出合理预测                         |

---

### 缺点

- It comes with a cost: the state values may not be accurately represented by the function.
- From a fundamental point of view, some information will certainly be lost when we use a lowdimensional vector to represent a high-dimensional dataset.
- Therefore, the function  approximation method enhances storage efficiency by sacrificing accuracy.

---

### 总结

* 表格法不能处理高维、连续状态；
* 函数近似方法（如线性模型、神经网络）允许我们泛化学习结果；
* 强化学习中的值函数学习可以类比为回归问题；
* 后续我们将介绍值函数近似中的目标函数、优化方法、以及 DQN 等典型算法。




## Part 2：原理 — 目标函数介绍、优化算法和函数选择、示例与分析

---

## 为什么要近似值函数？

* 当状态空间 $\mathcal{S}$ 太大或连续时，无法再为每个状态单独存储一个值；
* 所以我们使用一个参数函数 $\hat{V}(s; \theta)$ 或 $\hat{Q}(s, a; \theta)$ 来**学习一个通用表达式**；
* 然后使用优化算法不断更新参数 $\theta$，使其尽可能准确地拟合实际回报。

---

## 1. 值函数近似的目标函数

在强化学习中，值函数近似的目标是使估计值函数与“真实值函数”尽量接近。通常我们会设计一个**损失函数**并最小化它。

---

### 1.1 对于策略评估（预测任务）

给定策略 $\pi$，希望学习估计函数 $\hat{V}(s; \theta) \approx V_\pi(s)$

**目标函数：最小均方值误差（MSVE）**：

$$
J(\theta) = \mathbb{E}_{s \sim d_\pi} \left[ \left( V_\pi(s) - \hat{V}(s; \theta) \right)^2 \right]
$$

其中 $d_\pi(s)$ 是策略 $\pi$ 下的状态分布。

> 这是一个标准的**监督学习回归问题**，目标是使预测值尽量接近真实的 $V_\pi(s)$。


---

### 1.2 使用 Monte Carlo 方法的目标（全回报）

当环境允许完整 episode 采样时，可用 Monte Carlo 方法估计“真实”回报：

* 在 episode 结束后可计算：

  $$
  G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots = \sum_{k=0}^{T - t - 1} \gamma^k r_{t+1+k}
  $$
* 然后用 \$G\_t\$ 作为目标更新值函数：

  $$
  J(\theta) = \left( G_t - \hat{V}(s_t; \theta) \right)^2
  $$

**优点**：

* 是对真实期望回报的**无偏估计**；

**缺点**：

* 需要等待完整 episode 结束；
* 回报方差大，训练不稳定，收敛慢。


---

### 1.3 使用 TD 方法的目标函数（TD误差）

在没有 $V_\pi(s)$ 的精确值时，我们使用贝尔曼等式中的估计值作为“目标”。

**TD(0) 中的 TD误差定义为**：

$$
\delta_t = r_{t+1} + \gamma \hat{V}(s_{t+1}; \theta) - \hat{V}(s_t; \theta)
$$

然后使用它的平方作为损失函数进行优化：

$$
J(\theta) = \delta_t^2 = \left( r_{t+1} + \gamma \hat{V}(s_{t+1}; \theta) - \hat{V}(s_t; \theta) \right)^2
$$


![](./img/8_1.png)

---

### 两种误差对比

| 项目 | Monte Carlo 误差（全回报）      | TD 误差（一步 bootstrap）                          |
| -- |--------------------------| -------------------------------------------- |
| 目标 | $G_t \approx V_\pi(s_t)$ | $r_{t+1} + \gamma V(s_{t+1}) \approx V(s_t)$ |
| 优势 | 无偏估计（unbiased）           | 方差小，更新更快                                     |
| 劣势 | 方差大，需要完整 episode         | 有偏估计，但收敛快                                    |

---

## 2. 优化方法：如何更新参数 $\theta$

### 2.1 梯度下降（Gradient Descent）

目标函数是某种误差 $J(\theta)$，我们使用梯度下降进行参数更新：

$$
\theta \leftarrow \theta - \alpha \nabla_\theta J(\theta)
$$

对于 TD 误差的平方损失：

$$
J(\theta) = \left( r_{t+1} + \gamma \hat{V}(s_{t+1}; \theta) - \hat{V}(s_t; \theta) \right)^2
$$

其梯度为：

$$
\nabla_\theta J(\theta) = -2 \delta_t \nabla_\theta \hat{V}(s_t; \theta)
$$

更新公式为：

$$
\theta \leftarrow \theta + \alpha \cdot \delta_t \cdot \nabla_\theta \hat{V}(s_t; \theta)
$$

---

### 2.2 Stochastic Gradient Descent (SGD)

由于每次交互只能采集一个样本，所以只能使用 **SGD** 进行增量式更新：

$$
\theta \leftarrow \theta + \alpha \cdot \left( r_{t+1} + \gamma \hat{V}(s_{t+1}; \theta) - \hat{V}(s_t; \theta) \right) \cdot \nabla_\theta \hat{V}(s_t; \theta)
$$

---

## 3. 函数近似器的选择

---

### 3.1 线性函数近似（Linear Function Approximation）

$$
\hat{V}(s; \theta) = \theta^\top \phi(s)
$$

* 其中 $\phi(s)$ 是状态的特征向量；
* 优点：计算高效、理论成熟；
* 缺点：容量有限，表达能力弱。

---

### 3.2 非线性函数（如神经网络）

$$
\hat{V}(s; \theta) = \text{MLP}(s; \theta)
$$

* 使用神经网络对输入状态建模；
* 强大的表达能力，可用于高维状态；
* 需要更复杂的训练技术（如经验回放、稳定技巧等）；
* 代表算法：**DQN（Deep Q Network）**。

---

### 3.3 Tile Coding、Radial Basis Functions、Fourier Basis

这些是介于线性和深度网络之间的方案，在经典 RL 中也有广泛应用。

---

## 4. 示例与可视化分析

---

### 例子：Mountain Car（小车爬坡）

* 状态连续（位置 + 速度）；
* 使用表格方法不可行；
* 可使用神经网络或 tile coding 对值函数进行逼近；
* 学习目标：

  $$
  Q(s,a; \theta) \approx \text{期望回报}
  $$

通过更新参数 $\theta$，使函数输出逼近 TD 目标或 MC 目标。

---

## 总结

| 模块       | 内容                     |
| -------- | ---------------------- |
| 目标函数     | MSE、TD误差平方、Bellman误差   |
| 更新方法     | 梯度下降、SGD、增量更新          |
| 函数选择     | 线性模型、神经网络、tile coding等 |
| 精度/稳定性权衡 | MC 无偏但高方差，TD 有偏但低方差    |
| 应用       | 为深度强化学习（如 DQN）奠定基础     |






# Part 3：Sarsa 和 Q-learning 的函数逼近

> 目标：将 Sarsa 和 Q-learning 从表格方法扩展到使用函数逼近（Function Approximation）的方法中，支持高维、连续状态/动作空间。

---

## 一、为什么需要函数逼近？

在传统的强化学习中：

* 表格法：每个状态-动作对 $(s,a)$ 都单独存储一个值 $Q(s,a)$；
* 问题：当状态空间或动作空间很大或是连续的，无法枚举所有 $(s,a)$；

为此我们需要引入函数逼近器：

$$
Q(s,a; \theta) \approx Q^\pi(s,a) \quad \text{或} \quad Q(s,a; \theta) \approx Q^*(s,a)
$$

其中：

* $\theta$：是模型的可学习参数（比如神经网络权重）；
* $Q(s,a; \theta)$：是动作值函数的逼近表达。

---

## 二、核心思想

我们不再直接更新一个表格，而是通过最小化**时序差分误差（TD Error）**，用**梯度下降法**来更新参数 $\theta$。

---

## 三、Sarsa with Function Approximation（On-policy）


![](./img/8_2.png)

![](./img/8_3.png)

### 3.1 目标

逼近策略 $\pi$ 下的动作值函数 $Q^\pi(s,a)$，估计形式：

$$
\hat{Q}(s,a;\theta) \approx Q^\pi(s,a)
$$

### 3.2 更新方式（使用 TD 目标）

1. **TD目标（bootstrapped target）**：

$$
\text{Target}_t = r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}; \theta)
$$

2. **TD误差（TD error）**：

$$
\delta_t = \text{Target}_t - Q(s_t, a_t; \theta)
= r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}; \theta) - Q(s_t, a_t; \theta)
$$

3. **梯度更新**：

$$
\theta \leftarrow \theta + \alpha \cdot \delta_t \cdot \nabla_\theta Q(s_t, a_t; \theta)
$$

### 3.3 使用建议：

* Sarsa 是 **on-policy**：使用采样动作 $a_{t+1}$；
* 稳定性比 Q-learning 好，但收敛速度可能慢。

---

## 四、Q-learning with Function Approximation（Off-policy）

### 4.1 目标

逼近**最优动作值函数** $Q^*(s,a)$。

$$
\hat{Q}(s,a;\theta) \approx Q^*(s,a)
$$

### 4.2 更新方式（使用最大动作作为目标）

![](./img/8_4.png)

1. **TD目标（off-policy target）**：

$$
\text{Target}_t = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a'; \theta)
$$

2. **TD误差**：

$$
\delta_t = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a'; \theta) - Q(s_t, a_t; \theta)
$$

3. **梯度更新**：

$$
\theta \leftarrow \theta + \alpha \cdot \delta_t \cdot \nabla_\theta Q(s_t, a_t; \theta)
$$

### 4.3 特点：

* **off-policy**：目标策略是 greedy 策略；
* 高效探索（如 ε-greedy）+ 目标最优化（max）；
* 更容易发散（尤其搭配非线性逼近时，如深度网络）；

---

## 五、线性函数逼近：推导简化版

当我们使用线性函数：

$$
Q(s,a; \theta) = \theta^\top \phi(s,a)
$$

其中 $\phi(s,a) \in \mathbb{R}^d$ 是状态-动作对的特征向量。

* **梯度为**：

$$
\nabla_\theta Q(s,a; \theta) = \phi(s,a)
$$

* **更新公式（例如 Q-learning）简化为**：

$$
\theta \leftarrow \theta + \alpha \cdot \delta_t \cdot \phi(s,a)
$$

---

## 六、伪代码（以 Sarsa 为例）

```python
Initialize θ randomly
for each episode:
    s = initial state
    a = ε-greedy policy from Q(s, ·; θ)
    for each step t:
        Take action a, observe r, s'
        a' = ε-greedy policy from Q(s', ·; θ)
        δ = r + γ * Q(s', a'; θ) - Q(s, a; θ)
        θ ← θ + α * δ * ∇_θ Q(s, a; θ)
        s ← s', a ← a'
```

---

## 七、表格 vs 函数逼近：关键对比

| 特性      | 表格法         | 函数逼近                 |
| ------- | ----------- | -------------------- |
| 状态/动作空间 | 小，离散        | 可处理连续/高维             |
| 存储结构    | Q-table     | 参数向量 $\theta$        |
| 泛化能力    | 差，只能记忆      | 可泛化未见过的状态/动作         |
| 更新方式    | 显式更新 Q(s,a) | 通过梯度下降更新 θ           |
| 易收敛性    | 理论成熟，易分析    | 容易发散（特别是 Q-learning） |

---

## 八、为什么 Q-learning 易发散？（深度学习视角）

Q-learning 是 off-policy、基于 max 的方法，如果你使用非线性逼近器（如 DNN）：

* max 操作导致目标不连续；
* off-policy 更新带来分布不一致性；
* Bootstrap 使目标带噪声，错误会持续积累。

> 所以 DQN 采用了 **target network + experience replay** 来缓解这些问题。

---

## 总结：Sarsa vs Q-learning with Function Approximation

| 方法             | 策略类型       | 目标函数                      | 特点             |
| -------------- | ---------- | ------------------------- | -------------- |
| Sarsa(FA)      | On-policy  | $r + \gamma Q(s', a')$    | 更稳定，适用于策略评估/改进 |
| Q-learning(FA) | Off-policy | $r + \gamma \max Q(s',·)$ | 收敛快，适用于寻找最优策略  |





## Part 4：DQN — 基本原理、Experience Replay、代码与例子

---

###  一、背景：为什么需要 DQN？

Q-learning 能找到最优策略，但在面对**高维、连续状态空间**时：

* Q-table 无法存储所有状态；
* 使用神经网络作为 Q 函数逼近器是自然选择；
* 但直接使用 DNN 的 Q-learning 极易**发散**（如前所述）；

---

###  二、DQN（Deep Q-Network）的基本原理

DQN 是 Google DeepMind 在 2015 年提出的深度强化学习方法（Nature, 2015），成功在 Atari 游戏中超越人类水平。

#### 核心思想：

用一个深度神经网络 $Q(s,a; \theta)$ 来逼近最优 Q 函数 $Q^*(s,a)$。

#### 问题：Q-learning 的不稳定来源？

1. **bootstrap target** 带有估计误差；
2. **max 操作** 会放大估计偏差（max overestimation）；
3. 样本之间高度相关性（非 i.i.d.）；
4. 网络参数边训练边作为目标，导致目标快速漂移；

---

###  三、DQN 的两个关键机制

为了解决上面的问题，DQN 引入两个改进机制：

---

####  1）Experience Replay（经验回放）

* 将代理与环境交互获得的每个转移 $(s,a,r,s')$ 存储在一个 **回放缓冲区**（Replay Buffer）中；
* 每次训练时，从中\*\*随机采样一个小批量（mini-batch）\*\*进行更新；
* 优点：

  * 打破样本之间的相关性（增加 i.i.d. 假设的合理性）；
  * 更高的数据利用率（每条经验可用多次）；

---

####  2）Target Network（目标网络）

* 使用一个**固定的参数副本 $\theta^-$** 来计算目标：

  $$
  y = r + \gamma \max_{a'} Q(s', a'; \theta^-)
  $$
* 目标网络参数每隔一定步数 $C$ 才从主网络拷贝：

  $$
  \theta^- \leftarrow \theta
  $$
* 避免目标随梯度变化太快，导致训练不稳定。

---

###  四、DQN 算法流程（简化）

```python
Initialize Q-network with weights θ
Initialize target network with weights θ⁻ = θ
Initialize replay buffer D

for episode = 1 to N:
    s = initial state
    for t = 1 to max_steps:
        # ε-greedy action selection
        with prob ε: a = random
        else:        a = argmax_a Q(s,a; θ)

        # Execute action
        s', r, done = env.step(a)

        # Store transition
        D.append((s, a, r, s', done))

        # Sample random mini-batch from D
        for (s_j, a_j, r_j, s'_j, done_j) in batch:
            if done_j:
                y_j = r_j
            else:
                y_j = r_j + γ * max_a' Q(s'_j, a'; θ⁻)

            # Loss = MSE
            L = (Q(s_j, a_j; θ) - y_j)^2

            # Gradient descent step
            θ ← θ - α ∇_θ L

        # 每 C 步更新一次目标网络
        if step % C == 0:
            θ⁻ ← θ

        s ← s'
        if done: break
```

---

###  五、DQN 的损失函数

使用均方误差（MSE）损失：

$$
\mathcal{L}(\theta) = \mathbb{E}_{(s,a,r,s') \sim \mathcal{D}} \left[ \left( y - Q(s,a;\theta) \right)^2 \right]
$$

其中目标值：

$$
y =
\begin{cases}
r, & \text{if } s' \text{ is terminal} \\
r + \gamma \max_{a'} Q(s', a'; \theta^-), & \text{otherwise}
\end{cases}
$$

---

###  六、完整 DQN 系统结构图

```text
环境 --> [Replay Buffer] --> [Mini-batch] --> Q网络 + 目标网络 --> TD target --> Loss --> θ 更新
```

---

###  七、经验与注意事项

* **缓冲区大小**：通常设为 1e5 或 1e6；
* **mini-batch**：通常为 32 或 64；
* **目标网络更新频率**：如每 1000 步；
* **ε-greedy 策略**：初始 1.0，线性下降到 0.1；
* **收敛技巧**：使用 `Huber loss` 替代 MSE（更鲁棒）；

---

###  八、DQN 的扩展版本（附带参考）

| 方法                            | 特点说明                               |
| ----------------------------- | ---------------------------------- |
| Double DQN                    | 使用两套网络避免过高估计                       |
| Dueling DQN                   | 拆解 state value 和 advantage 更有效学习结构 |
| Prioritized Experience Replay | 优先采样 TD 误差大的样本加速学习                 |
| Rainbow DQN                   | 综合多项改进（上面三个 + NoisyNet + n-step 等） |

---

###  总结：DQN 的关键点

| 模块                | 功能描述                   |
| ----------------- | ---------------------- |
| Q(s,a; θ)         | 用深度网络近似 Q 值函数          |
| Experience Replay | 提高样本利用率，打破相关性          |
| Target Network    | 减少目标变化波动，提高训练稳定性       |
| TD目标              | 使用 bootstrapped 目标更新网络 |
| ε-greedy 策略       | 保证探索，防止策略陷入局部最优        |

---



## 本课总结

* 函数逼近是解决大规模或连续状态空间的关键
* TD目标和函数逼近结合，形成近似TD学习
* DQN利用深度神经网络和经验回放，极大提升了Q-learning的表现

# Q1 在值函数近似中，策略函数（policy）是如何参与的？


---

## 策略函数的定义与两类任务

在强化学习中，策略 $\pi(a|s)$ 是一个从状态到动作概率的映射：

* 在 **策略评估（prediction）任务** 中：已知策略 $\pi$，我们只想估计 $V_\pi(s)$ 或 $Q_\pi(s, a)$。
* 在 **策略优化（control）任务** 中：策略会随着值函数变化而迭代更新，最终趋近最优策略 $\pi^*$。

---

## 在值函数近似中策略的作用（分情况说明）

---

### 情况 1：**On-policy 策略评估（如 MC, TD, Sarsa）**

> 你有一个固定策略 $\pi$，目标是学习它的值函数。

#### 使用方式：

* 用策略 $\pi$ 与环境交互生成状态转移序列；
* 利用这些数据更新参数 $\theta$，逼近：

  $$
  \hat{V}(s; \theta) \approx V_\pi(s), \quad \text{或} \quad \hat{Q}(s,a; \theta) \approx Q_\pi(s,a)
  $$
* 其中 $\pi$ 也用于估计未来期望，例如：

  $$
  V_\pi(s) = \sum_a \pi(a|s) Q_\pi(s,a)
  $$

  或 TD target:

  $$
  r_{t+1} + \gamma \sum_a \pi(a|s_{t+1}) Q(s_{t+1}, a; \theta)
  $$

#### 总结：

* 策略用于：

  * 数据采样（行为）
  * 计算期望（如 bootstrapping）

---

### 情况 2：**Off-policy 学习（如 Q-learning, DQN）**

> 你想学习一个最优策略，但可以用其他策略（行为策略）收集数据。

#### 使用方式：

* 实际行为由某个行为策略（如 epsilon-greedy）产生；
* 目标是逼近最优策略的值函数：

  $$
  Q(s,a; \theta) \approx Q^*(s,a)
  $$
* 所以 TD 目标变成：

  $$
  r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a'; \theta)
  $$

#### 策略的作用：

* **目标策略**：取 $\arg\max_a Q(s,a)$，目标是学习最优策略；
* **行为策略**：如 $\epsilon$-greedy 生成样本，保证探索性；
* 行为策略 ≠ 目标策略 → **off-policy**

---

### 情况 3：**Actor-Critic 方法**

> 策略和值函数都使用函数逼近，并共同优化。

#### 使用方式：

* 策略函数是 $\pi(a|s; \phi)$，由参数 $\phi$ 控制；
* 值函数是 $V(s; \theta)$，由参数 $\theta$ 控制；
* 训练流程如下：

  1. 用 $\pi(a|s; \phi)$ 与环境交互；
  2. 用当前策略估算 TD 误差：

     $$
     \delta_t = r_{t+1} + \gamma V(s_{t+1}; \theta) - V(s_t; \theta)
     $$
  3. 用 $\delta_t$ 作为优势估计（advantage）更新策略参数：

     $$
     \nabla_\phi J(\phi) \propto \delta_t \nabla_\phi \log \pi(a_t|s_t; \phi)
     $$

#### 总结：

* 策略函数是模型的一部分（Actor）；
* 值函数是辅助函数（Critic）；
* 两者一起近似并优化强化学习目标。

---

## ✅ 总结对比

| 方法类型             | 策略的作用                     |
| ---------------- | ------------------------- |
| 策略评估（on-policy）  | 指定采样行为 + 参与计算 TD 目标中的期望   |
| 策略评估（off-policy） | 实际行为来自其他策略，但估计目标策略的值函数    |
| Q-learning/DQN   | 策略通过 $\max_a Q(s,a)$ 间接确定 |
| Actor-Critic     | 策略函数本身可学习、可逼近，与值函数共同训练    |



# Q2 如何直观理解时序差分？

---

## 一、名字含义：为什么叫“时序差分”？

“时序差分”这个名字来源于它的核心思想：

> **用相邻两个时间步的值函数估计之间的差值（即“差分”）来学习。**

### 核心公式（TD 误差）：

$$
\delta_t = r_{t+1} + \gamma V(s_{t+1}) - V(s_t)
$$

可以看到：

* **$V(s_t)$** 是当前状态的值函数；
* **$r_{t+1} + \gamma V(s_{t+1})$** 是下一个状态带来的“一阶更新目标”；
* 这两个量的差值，就叫做**时序差分误差（TD Error）**。

所以：

> 它不需要完整地走完整条轨迹，而是**从当前状态跳到下一状态时，就能进行一次差分估计更新**。

---

## 二、与“蒙特卡洛方法”的对比

| 方法       | 学习方式               | 所需信息                              | 更新时间          |
| -------- | ------------------ | --------------------------------- | ------------- |
| 蒙特卡洛（MC） | 从整条 episode 得到完整回报 | $G_t = r_{t+1} + r_{t+2} + \dots$ | episode 结束后更新 |
| 时序差分（TD） | 从当前一步 + 下一步状态估计回报  | $r_{t+1} + \gamma V(s_{t+1})$     | 每一步都能更新       |

### 关键理解：

* MC 是基于完整轨迹的**经验回报**；
* TD 是基于当前状态和下一状态的**局部差分估计**。

这就是为什么 TD 方法被认为是：

> **“更及时（temporal）”、“更高效”、“更可在线更新”的方法**。

---

## 三、一个直观类比

假设你要判断一个山坡的高度变化：

* **蒙特卡洛法**就像你从山脚走到山顶，才能测出海拔差；
* **时序差分法**就像你每走一步就用脚感受地形起伏，并据此不断更新对山坡整体走势的判断。

---

## 四、总结：TD 的直观特性

| 特性           | 说明                             |
| ------------ | ------------------------------ |
| “Temporal”   | 利用相邻两个时间步（状态）之间的信息更新           |
| “Difference” | 当前估计与 bootstrapped 目标之间的差值（误差） |
| 及时性（Online）  | 每一步即可更新，不必等待 episode 结束        |
| 高效性（数据利用）    | 能快速利用当前环境反馈进行学习                |

---

## 总结一句话：

> “时序差分”法就是用当前奖励和下一状态的估计值，去逼近当前状态值的好坏，从而进行实时、逐步的值函数学习。

